In [23]:
from underthesea import word_tokenize
import pandas as pd
import openpyxl
# Đọc file Excel
file_path = "data.xlsx"
df = pd.read_excel(file_path)

# Hiển thị dữ liệu
print(df[['Tên địa điểm', 'Mô tả']])

          Tên địa điểm                                      Mô tả
0         Vịnh Hạ Long     Kỳ quan thiên nhiên nổi tiếng thế giới
1         Chùa Một Cột       Ngôi chùa độc đáo có lịch sử lâu đời
2  Phong Nha - Kẻ Bàng  Vườn quốc gia với các hang động nổi tiếng
3             Chùa Dơi          Ngôi chùa có nhiều dơi và lâu đời
4               Đà Lạt      Thành phố ngàn hoa với khí hậu mát mẻ


In [24]:
from underthesea import word_tokenize
import pandas as pd
from collections import Counter

# Đọc file Excel
file_path = "data.xlsx"
df = pd.read_excel(file_path)

# Bước 1: Tách từ trong cột "Mô tả"
all_words = []
for description in df['Mô tả']:
    # Tokenize mỗi câu mô tả và thêm vào danh sách
    words = word_tokenize(str(description), format='text').split()
    all_words.extend(words)

# Bước 2: Đếm tần suất xuất hiện của từng từ
word_freq = Counter(all_words)

# Bước 3: Tạo danh sách stop words dựa trên tần suất
# Chọn các từ xuất hiện quá nhiều lần làm stop words (ví dụ: tần suất > 10)
vietnamese_stopwords = [word for word, freq in word_freq.items() if freq > 10]

# Hiển thị danh sách stop words
print("Danh sách Vietnamese Stopwords:")
print(vietnamese_stopwords)


Danh sách Vietnamese Stopwords:
[]


In [25]:
# Hàm tách từ và loại bỏ stop words
def preprocess_text(text):
    # Tách từ
    tokens = word_tokenize(text, format="text").split()
    # Loại bỏ stop words
    tokens = [word for word in tokens if word.lower() not in vietnamese_stopwords]
    return ' '.join(tokens)

# Áp dụng tiền xử lý lên từng mô tả sách
df['Mô tả sau tách'] = df['Mô tả'].apply(preprocess_text)
print(df[['Tên địa điểm', 'Mô tả sau tách']])

          Tên địa điểm                             Mô tả sau tách
0         Vịnh Hạ Long     Kỳ_quan thiên_nhiên nổi_tiếng thế_giới
1         Chùa Một Cột       Ngôi chùa độc_đáo có lịch_sử lâu_đời
2  Phong Nha - Kẻ Bàng  Vườn_quốc_gia với các hang_động nổi_tiếng
3             Chùa Dơi          Ngôi chùa có nhiều dơi và lâu_đời
4               Đà Lạt      Thành_phố ngàn hoa với khí_hậu mát_mẻ


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Tạo TF-IDF matrix từ mô tả đã được xử lý
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['Mô tả sau tách'])

# Kiểm tra ma trận TF-IDF
print("TF-IDF Matrix shape:", tfidf_matrix.shape)

TF-IDF Matrix shape: (5, 22)


In [27]:
from sklearn.metrics.pairwise import cosine_similarity

# Tính độ tương tự cosine giữa các sách
place_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Hàm gợi ý sách
def recommend_books(place_title, df, similarity_matrix, num_recommendations=3):
    # Tìm chỉ số của sách trong DataFrame
    book_idx = df[df['Tên địa điểm'] == place_title].index[0]

    # Tính toán độ tương tự giữa sách này và các sách khác
    similarity_scores = list(enumerate(similarity_matrix[book_idx]))

    # Sắp xếp các sách theo điểm tương tự, loại bỏ sách đã chọn
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Lấy danh sách sách gợi ý (trừ sách đã chọn)
    recommended_books_indices = [i[0] for i in similarity_scores[1:num_recommendations+1]]

    # Trả về tên các sách gợi ý
    return df['Tên địa điểm'].iloc[recommended_books_indices]



In [28]:
recommendations = recommend_books("Chùa Một Cột", df, place_similarity, num_recommendations=3)
print("Recommended place: \n", recommendations)


Recommended place: 
 3               Chùa Dơi
0           Vịnh Hạ Long
2    Phong Nha - Kẻ Bàng
Name: Tên địa điểm, dtype: object
